In [44]:
import io
from typing import Set

import netaddr

import pandas as pd

import aiohttp

from rpki_analysis.delegated_stats import read_delegated_extended_stats, PytriciaLookup, extract_resource

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/ripe/stats/nro-stats/latest/nro-delegated-stats') as resp:
        df = read_delegated_extended_stats(io.StringIO(await resp.text()))
        # we want textual resources
        df['resource'] = df['resource'].apply(str)

In [3]:
df_asn = df[df.afi == 'asn'].copy()

In [4]:
# pyinterval may not be installable on 3.13
from interval import interval

def interval_from_length(row) -> interval:
    return interval([row.raw_resource, int(row.raw_resource) + row.length])

In [5]:
df_asn['resource'] = df_asn.apply(interval_from_length, axis=1)

In [6]:
#df_asn[['resource', 'opaque_id'], df_asn['opaque_id'] == 'ripe'].groupby(['opaque_id']).apply(display)

In [7]:
df_asn[['opaque_id', 'rir', 'length']]\
    .groupby(['opaque_id', 'rir'])\
    .sum().reset_index()\
    .nlargest(columns=['length'], n=50)

,opaque_id,rir,length
474213,iana,iana,4199803376
474219,ietf,iana,95033875
474233,ripencc,ripencc,6197
330871,A91E5D61,apnic,4268
313021,A918EDB2,apnic,3341
311179,A9186214,apnic,3261
370058,arin,arin,2053
370051,apnic,apnic,1789
474226,lacnic,lacnic,1721
299173,A914BC7A,apnic,1654


In [8]:
df_top_asn_holders = df_asn[['opaque_id', 'rir', 'length']]\
    .groupby(['opaque_id', 'rir'])\
    .sum().reset_index()\
    .nlargest(columns=['length'], n=50)

In [9]:
df_top = df_top_asn_holders[~df_top_asn_holders.opaque_id.isin(set(['arin', 'apnic', 'ripencc', 'lacnic','ietf', 'iana']))].copy()

In [10]:
df[df.afi!='asn'].join(df_top.set_index(['opaque_id']), on='opaque_id', how='inner', rsuffix='r_').nlargest(columns=['lengthr_'], n=100)

,rir,country,afi,raw_resource,length,date,status,opaque_id,category,resource,rirr_,lengthr_
135458,apnic,CN,ipv4,58.198.0.0,131072,2006-07-11 00:00:00+00:00,assigned,A91E5D61,e-stats,58.198.0.0-58.199.255.255,apnic,4268
202863,apnic,CN,ipv4,113.54.0.0,131072,2008-10-22 00:00:00+00:00,assigned,A91E5D61,e-stats,113.54.0.0-113.55.255.255,apnic,4268
203214,apnic,CN,ipv4,114.212.0.0,131072,2008-07-04 00:00:00+00:00,assigned,A91E5D61,e-stats,114.212.0.0-114.213.255.255,apnic,4268
203215,apnic,CN,ipv4,114.214.0.0,65536,2008-07-04 00:00:00+00:00,assigned,A91E5D61,e-stats,114.214.0.0-114.214.255.255,apnic,4268
203222,apnic,CN,ipv4,115.24.0.0,262144,2008-07-04 00:00:00+00:00,assigned,A91E5D61,e-stats,115.24.0.0-115.27.255.255,apnic,4268
...,...,...,...,...,...,...,...,...,...,...,...,...
395866,apnic,IN,ipv6,2001:df3:6640::,48,2024-01-24 00:00:00+00:00,assigned,A918EDB2,e-stats,2001:df3:6640::/48,apnic,3341
410499,apnic,IN,ipv6,2001:df6:c700::,48,2018-09-24 00:00:00+00:00,assigned,A918EDB2,e-stats,2001:df6:c700::/48,apnic,3341
420633,apnic,IN,ipv6,2400:3b01::,32,2022-06-06 00:00:00+00:00,assigned,A918EDB2,e-stats,2400:3b01::/32,apnic,3341
428505,apnic,IN,ipv6,2400:f080::,32,2014-04-30 00:00:00+00:00,assigned,A918EDB2,e-stats,2400:f080::/32,apnic,3341


In [11]:
df_top.dtypes

opaque_id      object
rir          category
length          int64
dtype: object

In [12]:
df_asn_count = df_asn[['opaque_id', 'length']]\
    .groupby(['opaque_id'])\
    .sum().reset_index()\
    .rename(columns={'length': 'num_asns'})

In [13]:
df_ext_stats_with_asn_count = df[df.afi!='asn'].join(df_asn_count.set_index(['opaque_id']), on='opaque_id', how='inner')

In [14]:
#df_ext_stats_with_asn_count.to_csv('/tmp/delegated_extended_asn-count.csv')

Now get the csvext from routinator to get the certificate URLs.

In [15]:
from rpki_analysis.rpki_client import read_dump_url

df_dump = await read_dump_url("https://console.rpki-client.org/dump.json")

In [ ]:
df_dump.keys()

In [26]:
class AiaLookup(PytriciaLookup[str]):
    """
    Just find the RIR responsible for the range
    """

    def __init__(self, data: pd.DataFrame) -> None:
        super().__init__()
        assert set(data.keys()) >= set(
            [
                "aia",
                "aki",
                "cert_issuer",
                "cert_serial",
                "expires",
                "file",
                "hash_id",
                "sia",
                "signing_time",
                "ski",
                "type",
                "valid_since",
                "valid_until",
                "validation",
                "prefix",
                "asn",
                "max_length"
            ]
        )
        data[['prefix', 'aia']].groupby(["aia"]).apply(self.__build_trie)

    def __build_trie(self, rows: pd.Series) -> None:
        """Build trie entries for the groups of rows.
        @precondition grouped by prefix
        """
        aias = rows.aia.unique()
        assert len(aias) == 1
        aia = aias[0]

        cidrs = netaddr.cidr_merge([row.prefix for (_, row) in rows.iterrows()])
        for cidr in cidrs:
            if cidr.version == 4:
                self.trie4[str(cidr)] = aia
            else:
                self.trie6[str(cidr)] = aia

In [27]:
lookup = AiaLookup(df_dump)

In [28]:
lookup['130.89.0.0/16']

'rsync://rpki.ripe.net/repository/DEFAULT/s-98dqs-aXHY-HMC3fksxgmISeo.cer'

In [32]:
next(iter(set(lookup.children("193.0.0.0/20"))))

'rsync://rpki.ripe.net/repository/DEFAULT/iBeq_sYcOmsCS2qfVP-3ZMo9-SI.cer'

In [19]:
df_dump

,prefix,asn,max_length,file,hash_id,type,ski,cert_issuer,cert_serial,aki,aia,sia,signing_time,valid_since,valid_until,expires,validation,error
0,146.19.0.0/24,8888,24,0.sb/repo/sb/1/3134362e31392e302e302f32342d323...,2uIAnSnrLBcdTbY18RSUASHhfsWX4Y/Rk+aTHoBVkFs=,roa,51:6C:7C:33:26:FA:DB:5E:5A:83:6E:F5:EC:61:CF:D...,/CN=917b088922d0f33feca164e08554ec992daa39c5,64B596C46382BE15A92A14ACA8CEB20237A178CF,91:7B:08:89:22:D0:F3:3F:EC:A1:64:E0:85:54:EC:9...,rsync://rpki.ripe.net/repository/DEFAULT/kXsIi...,rsync://0.sb/repo/sb/1/3134362e31392e302e302f3...,1695377983,1695377683,1726827583,1.720134e+09,OK,NaN
1,146.19.145.0/24,8888,24,0.sb/repo/sb/1/3134362e31392e3134352e302f32342...,pJY2pZUTqfU+Ti3Kj0lT3F8b5Ly3whUMcyUs8w+kM1c=,roa,63:CF:21:F5:BF:65:CB:1C:71:19:6B:CE:F2:78:CB:B...,/CN=917b088922d0f33feca164e08554ec992daa39c5,5671772B457AC4416E8E60096694727C4544B89D,91:7B:08:89:22:D0:F3:3F:EC:A1:64:E0:85:54:EC:9...,rsync://rpki.ripe.net/repository/DEFAULT/kXsIi...,rsync://0.sb/repo/sb/1/3134362e31392e3134352e3...,1700322767,1700322467,1731772367,1.720134e+09,OK,NaN
2,2a11:3:500::/40,43357,48,0.sb/repo/sb/1/326131313a333a3530303a3a2f34302...,g6rHHtxHEQXiiuR5VApu6yqB6eXHYU4g6Qk/mQzm6Nw=,roa,9E:92:23:C2:1E:ED:74:1B:51:DC:E7:C6:FB:CE:00:A...,/CN=917b088922d0f33feca164e08554ec992daa39c5,24468E2595C49382EFC144AE5B2194D6FF21F443,91:7B:08:89:22:D0:F3:3F:EC:A1:64:E0:85:54:EC:9...,rsync://rpki.ripe.net/repository/DEFAULT/kXsIi...,rsync://0.sb/repo/sb/1/326131313a333a3530303a3...,1706181005,1706180705,1737630605,1.720134e+09,OK,NaN
3,2a11:3::/32,8888,48,0.sb/repo/sb/1/326131313a333a3a2f33322d3438203...,3VAInAQMbxff9RItF+z5mGPTlyTzFjs85Hm7aO0A6CI=,roa,41:F2:9A:7C:5F:4A:C9:64:E0:19:96:E7:81:DF:2C:7...,/CN=917b088922d0f33feca164e08554ec992daa39c5,28B5BDF7B42E16955E0B924C4BA88B04C82DAD59,91:7B:08:89:22:D0:F3:3F:EC:A1:64:E0:85:54:EC:9...,rsync://rpki.ripe.net/repository/DEFAULT/kXsIi...,rsync://0.sb/repo/sb/1/326131313a333a3a2f33322...,1695377984,1695377684,1726827584,1.720134e+09,OK,NaN
4,2a11:4::/32,8888,48,0.sb/repo/sb/1/326131313a343a3a2f33322d3438203...,nthjGprB29bS7F0/pVzJClj3P0tqbP/gPezAeajZs6A=,roa,B3:3D:56:75:B3:C2:D2:8C:32:65:E4:62:71:C4:19:D...,/CN=917b088922d0f33feca164e08554ec992daa39c5,68EC018671D2E29FFFCA7C0191BF6395B452A8A6,91:7B:08:89:22:D0:F3:3F:EC:A1:64:E0:85:54:EC:9...,rsync://rpki.ripe.net/repository/DEFAULT/kXsIi...,rsync://0.sb/repo/sb/1/326131313a343a3a2f33322...,1695377985,1695377685,1726827585,1.720134e+09,OK,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581212,2a13:1801:180::/43,151464,48,rsync.rpki.tianhai.link/repo/TianhaiRpki/0/326...,kghmO1MItu7ETLt8/ETjEoas52eeGmwEP7VXy5Y5fGI=,roa,48:04:F9:5D:F9:E2:B4:D8:62:F3:2C:59:91:AC:40:0...,/CN=3cdec27384aade5ca0809fd6f16d2bca18beb659,317E103A00DE1085046E8C663617DAF3876B77BF,3C:DE:C2:73:84:AA:DE:5C:A0:80:9F:D6:F1:6D:2B:C...,rsync://rpki.ripe.net/repository/DEFAULT/PN7Cc...,rsync://rsync.rpki.tianhai.link/repo/TianhaiRp...,1717850893,1717850593,1749300493,1.720080e+09,OK,NaN
581213,2a13:1802::/36,209306,44,rsync.rpki.tianhai.link/repo/TianhaiRpki/0/326...,rjSuK2x5LJ9JpoRZOm9Ea1uvdttRpcujWNO9Yf0KkJE=,roa,18:F3:F9:10:F1:79:DA:7B:2C:66:C7:97:2B:BD:B2:7...,/CN=3cdec27384aade5ca0809fd6f16d2bca18beb659,7D4BE6A89EA90C6B6FF45E1189B45950525F72A9,3C:DE:C2:73:84:AA:DE:5C:A0:80:9F:D6:F1:6D:2B:C...,rsync://rpki.ripe.net/repository/DEFAULT/PN7Cc...,rsync://rsync.rpki.tianhai.link/repo/TianhaiRp...,1710926954,1710926654,1742376554,1.720080e+09,OK,NaN
581214,103.150.172.0/23,38008,24,rsync.rpki.tianhai.link/repo/TianhaiRpki/3/313...,PVDMOTIHc0Ci+Izu7LqmiHEXNyVdYhr5+8jlMIJ0/ZQ=,roa,82:60:E4:A5:0E:23:92:4C:93:5E:5F:AD:B9:1B:52:D...,/CN=A91F56750000/serialNumber=459D2B834A3BA08C...,3E1BF26A62699967487070C0B511E169AE129D1D,45:9D:2B:83:4A:3B:A0:8C:0A:A6:7C:6E:57:81:46:A...,rsync://rpki.apnic.net/repository/B527EF581D66...,rsync://rsync.rpki.tianhai.link/repo/TianhaiRp...,1714826400,1714826100,1746276000,1.720095e+09,OK,NaN
581215,103.150.172.0/23,4842,24,rsync.rpki.tianhai

In [78]:
def lookup_all_cer(row) -> Set[str]:
    res = set()
    resource = extract_resource(row)
    # print(type(resource), resource)
    match resource:
        case netaddr.ip.IPRange():
            # print(resource)
            for cidr in resource.cidrs():
                res |= set(lookup.children(cidr))
        case netaddr.ip.IPNetwork():
            # print(resource)
            for cidr in resource.iter_cidrs():
                res |= set(lookup.children(cidr))

    if res:
        return ", ".join(res)
    return None
    

In [79]:
df_ip = df[df.afi != 'asn'].copy()

In [ ]:
df_ip['cert'] = df_ip.apply(lookup_all_cer, axis=1)

In [ ]:
df_ip

In [70]:
df_ip.loc[~df_ip.cert.isna(), set(df_ip.keys()) - set(['resource'])]

/var/folders/m4/qtvb8ff50ng0yw7xyt7t0n9c0000gp/T/ipykernel_30933/1922063749.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_ip.loc[~df_ip.cert.isna(), set(df_ip.keys()) - set(['resource'])]


,status,cert,category,date,opaque_id,length,rir,afi,country,raw_resource
